In [2]:
#coding:utf-8
#导入warnings包，利用过滤器来实现忽略警告语句。
import warnings
warnings.filterwarnings('ignore')

# GBDT
from sklearn.ensemble import GradientBoostingRegressor
# XGBoost
import xgboost as xgb
# LightGBM
import lightgbm as lgb

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
import pickle
import multiprocessing
from sklearn.preprocessing import StandardScaler
ss = StandardScaler() 
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC,LinearRegression,LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest

plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [3]:
## 先导入之前处理好的数据
train_path = '/Users/gongbin/Documents/数据竞赛/Datawhale/team-learning/数据竞赛（房租预测）/Data/train_data.csv'
test_path = '/Users/gongbin/Documents/数据竞赛/Datawhale/team-learning/数据竞赛（房租预测）/Data/test_a.csv'
data_train_origin = pd.read_csv(train_path)
data_train = data_train_origin.copy()
data_train['Type'] = 'Train'
data_test_origin = pd.read_csv(test_path)
data_test = data_test_origin.copy()
data_test['Type'] = 'Test'
data_all = pd.concat([data_train, data_test], ignore_index=True)

In [4]:
#对数据进行处理
def preprocessingData(data):
    # 填充缺失值
    data['rentType'][data['rentType'] == '--'] = '未知方式'  #进行填充
    
    # 转换object类型数据
    columns = ['rentType','communityName','houseType', 'houseFloor', 'houseToward', 'houseDecoration',  'region', 'plate']
    
    for feature in columns:
        data[feature] = LabelEncoder().fit_transform(data[feature])

    # 将buildYear列转换为整型数据
    buildYearmean = pd.DataFrame(data[data['buildYear'] != '暂无信息']['buildYear'].mode())  #用平均数填充
    data.loc[data[data['buildYear'] == '暂无信息'].index, 'buildYear'] = buildYearmean.iloc[0, 0]
    data['buildYear'] = data['buildYear'].astype('int')  #年份转成整数

    # 处理pv和uv的空值
    data['pv'].fillna(data['pv'].mean(), inplace=True)
    data['uv'].fillna(data['uv'].mean(), inplace=True)
    data['pv'] = data['pv'].astype('int')
    data['uv'] = data['uv'].astype('int')

    # 分割交易时间
    def month(x):
        month = int(x.split('/')[1])
        return month
    def year(x):
        year = int(x.split('/')[0])
        return year
    data['month'] = data['tradeTime'].apply(lambda x: month(x))
    data['year'] = data['tradeTime'].apply(lambda x: year(x))
    
    # 去掉部分特征
    data.drop('city', axis=1, inplace=True)
    data.drop('tradeTime', axis=1, inplace=True)
    data.drop('ID', axis=1, inplace=True)
    return data

data_train = preprocessingData(data_train)

In [5]:
# clean data
def IF_drop(train):
    IForest = IsolationForest(contamination=0.01)
    IForest.fit(train["tradeMoney"].values.reshape(-1,1))
    y_pred = IForest.predict(train["tradeMoney"].values.reshape(-1,1))
    drop_index = train.loc[y_pred==-1].index
    print(drop_index)
    train.drop(drop_index,inplace=True)
    return train

data_train = IF_drop(data_train)

/Users/gongbin/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


Int64Index([   62,    69,   128,   131,   246,   261,   266,   297,   308,
              313,
            ...
            39224, 39228, 39319, 39347, 39352, 39434, 39563, 41080, 41083,
            41233],
           dtype='int64', length=403)


In [6]:
def dropData(train):  
    # 丢弃部分异常值
    train = train[train.area <= 200]
    train = train[(train.tradeMoney <=16000) & (train.tradeMoney >=700)]
    train.drop(train[(train['totalFloor'] == 0)].index, inplace=True)
    return train  
#数据集异常值处理
data_train = dropData(data_train)

In [9]:
def newfeature(data):


    # 将houseType转为'Room'，'Hall'，'Bath'    用于判断单个房间数量对结果的影响
    '''
    def Room(x):
        Room = int(x.split('室')[0])
        return Room
    def Hall(x):
        Hall = int(x.split("室")[1].split("厅")[0])
        return Hall
    def Bath(x):
        Bath = int(x.split("室")[1].split("厅")[1].split("卫")[0])
        return Bath

    data['Room'] = data['houseType'].apply(lambda x: Room(x))
    data['Hall'] = data['houseType'].apply(lambda x: Hall(x))
    data['Bath'] = data['houseType'].apply(lambda x: Bath(x))
    data['Room_Bath'] = (data['Bath']+1) / (data['Room']+1)
    # 填充租房类型    根据
    '''



    # data['day'] = data['tradeTime'].apply(lambda x: day(x))# 结果变差
    #     data['pv/uv'] = data['pv'] / data['uv']
    #     data['房间总数'] = data['室'] + data['厅'] + data['卫']

    # 合并部分配套设施特征
    data['trainsportNum'] = 5 * data['subwayStationNum'] / data['subwayStationNum'].mean() + data['busStationNum'] / \
                                                                                             data[
                                                                                                 'busStationNum'].mean()
    data['all_SchoolNum'] = 2 * data['interSchoolNum'] / data['interSchoolNum'].mean() + data['schoolNum'] / data[
        'schoolNum'].mean() \
                            + data['privateSchoolNum'] / data['privateSchoolNum'].mean()
    data['all_hospitalNum'] = 2 * data['hospitalNum'] / data['hospitalNum'].mean() + \
                              data['drugStoreNum'] / data['drugStoreNum'].mean()
    data['all_mall'] = data['mallNum'] / data['mallNum'].mean() + \
                       data['superMarketNum'] / data['superMarketNum'].mean()
    data['otherNum'] = data['gymNum'] / data['gymNum'].mean() + data['bankNum'] / data['bankNum'].mean() + \
                       data['shopNum'] / data['shopNum'].mean() + 2 * data['parkNum'] / data['parkNum'].mean()

    data.drop(['subwayStationNum', 'busStationNum',
               'interSchoolNum', 'schoolNum', 'privateSchoolNum',
               'hospitalNum', 'drugStoreNum', 'mallNum', 'superMarketNum', 'gymNum', 'bankNum', 'shopNum', 'parkNum'],
              axis=1, inplace=True)
    # 提升0.0005
    
#     data['houseType_1sumcsu']=data['Bath'].map(lambda x:str(x))+data['month'].map(lambda x:str(x))
#     data['houseType_2sumcsu']=data['Bath'].map(lambda x:str(x))+data['communityName']
#     data['houseType_3sumcsu']=data['Bath'].map(lambda x:str(x))+data['plate']
    
    #data.drop('houseType', axis=1, inplace=True)
    #data.drop('tradeTime', axis=1, inplace=True)
    
    data["area"] = data["area"].astype(int)


    # categorical_feats = ['rentType', 'houseFloor', 'houseToward', 'houseDecoration', 'communityName','region', 'plate']
    categorical_feats = ['rentType', 'houseFloor', 'houseToward', 'houseDecoration',  'region', 'plate','cluster']

    return data, categorical_feats
data_train,categorical_feats = newfeature(data_train)

In [10]:
data_train_back = data_train.copy()

## 后续开始使用模型训练

In [11]:
data_train = data_train_back.copy()

In [12]:
## 导入基础库
from __future__ import print_function
import lightgbm as lgb
import sklearn
import numpy
import hyperopt
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
import colorama
import numpy as np

N_HYPEROPT_PROBES = 500
HYPEROPT_ALGO = tpe.suggest  #  tpe.suggest OR hyperopt.rand.suggest

In [13]:
colorama.init()  #colorama可以让终端显示不同颜色

# ---------------------------------------------------------------------

def get_lgb_params(space):    #设置lgb的不同参数，在space的搜索框架内
    lgb_params = dict()
    lgb_params['boosting_type'] = space['boosting_type'] if 'boosting_type' in space else 'gbdt'
    lgb_params['objective'] = 'regression'
    lgb_params['metric'] = 'rmse'
    lgb_params['learning_rate'] = space['learning_rate']
    lgb_params['num_leaves'] = int(space['num_leaves'])
    lgb_params['min_data_in_leaf'] = int(space['min_data_in_leaf'])
    lgb_params['min_sum_hessian_in_leaf'] = space['min_sum_hessian_in_leaf']
    lgb_params['max_depth'] = -1
    lgb_params['lambda_l1'] = space['lambda_l1'] if 'lambda_l1' in space else 0.0
    lgb_params['lambda_l2'] = space['lambda_l2'] if 'lambda_l2' in space else 0.0
    lgb_params['max_bin'] = int(space['max_bin']) if 'max_bin' in space else 256
    lgb_params['feature_fraction'] = space['feature_fraction']
    lgb_params['bagging_fraction'] = space['bagging_fraction']
    lgb_params['bagging_freq'] = int(space['bagging_freq']) if 'bagging_freq' in space else 1
    lgb_params['nthread'] = 4
    return lgb_params

In [14]:
obj_call_count = 0
cur_best_score = 0 # 0 or np.inf
log_path = '/Users/gongbin/Documents/数据竞赛/Datawhale/team-learning/数据竞赛（房租预测）/Data/lgb-hyperopt-log.txt'
log_writer = open(log_path, 'w' )

In [19]:
data_train.head()

,area,rentType,houseType,houseFloor,totalFloor,houseToward,houseDecoration,communityName,region,plate,...,lookNum,tradeMoney,Type,month,year,trainsportNum,all_SchoolNum,all_hospitalNum,all_mall,otherNum
0,68,2,12,1,16,6,0,50,0,63,...,0,2000.0,Train,11,2018,6.276900,0.737845,0.668611,0.743708,2.249440
1,125,2,28,0,14,6,2,129,1,48,...,1,2000.0,Train,12,2018,2.716272,0.537449,0.926699,1.250894,2.534403
2,132,2,28,1,32,6,0,178,1,49,...,1,16000.0,Train,12,2018,5.535024,2.557029,1.799363,0.999589,3.517453
3,57,2,4,0,17,6,3,312,1,50,...,9,1600.0,Train,12,2018,4.541916,4.099446,5.125212,1.727814,3.853332
4,129,2,29,1,2,6,1,1256,2,43,...,0,2900.0,Train,11,2018,0.749075,0.310066,0.361324,0.727166,1.934998


In [20]:
Y_train = data_train.pop('tradeMoney')
X_train = data_train


In [23]:
X_train.pop('Type')

0        Train
1        Train
2        Train
3        Train
4        Train
         ...  
41435    Train
41436    Train
41437    Train
41438    Train
41439    Train
Name: Type, Length: 40215, dtype: object

In [24]:
def objective(space):
    global obj_call_count, cur_best_score
    golbal X_train, Y_train

    obj_call_count += 1

    print('\nLightGBM objective call #{} cur_best_score={:7.5f}'.format(obj_call_count,cur_best_score) )

    lgb_params = get_lgb_params(space)  #获得参数列表

    sorted_params = sorted(space.items(), key=lambda z: z[0])
    params_str = str.join(' ', ['{}={}'.format(k, v) for k, v in sorted_params])
    print('Params: {}'.format(params_str) )
    
    kf = KFold(n_splits=3, shuffle=True, random_state=0)
    out_of_fold = np.zeros(len(X_train))
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
        D_train = lgb.Dataset(X_train.iloc[train_idx], label=Y_train[train_idx])
        D_val = lgb.Dataset(X_train.iloc[val_idx], label=Y_train[val_idx])
        # Train
        num_round = 10000
        clf = lgb.train(lgb_params,
                           D_train,
                           num_boost_round=num_round,
                           # metrics='mlogloss',
                           valid_sets=D_val,
                           # valid_names='val',
                           # fobj=None,
                           # feval=None,
                           # init_model=None,
                           # feature_name='auto',
                           # categorical_feature='auto',
                           early_stopping_rounds=200,
                           # evals_result=None,
                           verbose_eval=False,
                           # learning_rates=None,
                           # keep_training_booster=False,
                           # callbacks=None
                           )
        # predict
        nb_trees = clf.best_iteration
        val_loss = clf.best_score['valid_0']
        print('nb_trees={} val_loss={}'.format(nb_trees, val_loss))
        out_of_fold[val_idx] = clf.predict(X_train.iloc[val_idx], num_iteration=nb_trees)
        score = r2_score(out_of_fold, Y_train)

    print('val_r2_score={}'.format(score))

    log_writer.write('score={} Params:{} nb_trees={}\n'.format(score, params_str, nb_trees ))
    log_writer.flush()

    if score>cur_best_score:
        cur_best_score = score
        print(colorama.Fore.GREEN + 'NEW BEST SCORE={}'.format(cur_best_score) + colorama.Fore.RESET)
    return {'loss': -score, 'status': STATUS_OK}



SyntaxError: invalid syntax (<ipython-input-24-51fd5fde59c6>, line 3)

In [25]:
space ={
        'num_leaves': hp.quniform ('num_leaves', 10, 100, 1),
        'min_data_in_leaf':  hp.quniform ('min_data_in_leaf', 10, 100, 1),
        'feature_fraction': hp.uniform('feature_fraction', 0.75, 1.0),
        'bagging_fraction': hp.uniform('bagging_fraction', 0.75, 1.0),
        'learning_rate': hp.uniform('learning_rate', 0, 0.01),
#         'learning_rate': hp.loguniform('learning_rate', -5.0, -2.3),
        'min_sum_hessian_in_leaf': hp.loguniform('min_sum_hessian_in_leaf', 0, 2.3),
        'max_bin': hp.quniform ('max_bin', 88, 200, 1),
        'bagging_freq': hp.quniform ('bagging_freq', 1, 15, 1),
        'lambda_l1': hp.uniform('lambda_l1', 0, 10 ),
        'lambda_l2': hp.uniform('lambda_l2', 0, 10 ),
       }

trials = Trials()
best = hyperopt.fmin(fn=objective,
                     space=space,
                     algo=HYPEROPT_ALGO,
                     max_evals=N_HYPEROPT_PROBES,
                     trials=trials,
                     verbose=1)

print('-'*50)
print('The best params:')
print( best )
print('\n\n')

                                                       
LightGBM objective call #3 cur_best_score=0.00000
Params: bagging_fraction=0.7600202470526277 bagging_freq=1.0 feature_fraction=0.8603765313417686 lambda_l1=4.336712854330386 lambda_l2=6.578510252633993 learning_rate=0.00785100974426483 max_bin=178.0 min_data_in_leaf=34.0 min_sum_hessian_in_leaf=1.9256322894467406 num_leaves=87.0
nb_trees=256 val_loss=OrderedDict([('rmse', 2511.4287657658656)])
nb_trees=217 val_loss=OrderedDict([('rmse', 2556.392157761859)])
nb_trees=222 val_loss=OrderedDict([('rmse', 2578.9320844564654)])
val_r2_score=-54.21933911397814                        
                                                                                  
LightGBM objective call #4 cur_best_score=0.00000
Params: bagging_fraction=0.7949270954837502 bagging_freq=12.0 feature_fraction=0.9794080784197502 lambda_l1=2.7361697517290064 lambda_l2=2.081264315598502 learning_rate=0.004589984719741015 max_bin=89.0 min_data_in_leaf=46.0 m

LightGBM objective call #14 cur_best_score=0.00000
Params: bagging_fraction=0.9151201715146451 bagging_freq=11.0 feature_fraction=0.7745356284772779 lambda_l1=2.8060605320855947 lambda_l2=7.852083246963203 learning_rate=0.0006838591653925997 max_bin=136.0 min_data_in_leaf=49.0 min_sum_hessian_in_leaf=1.6466244906572336 num_leaves=18.0
nb_trees=7397 val_loss=OrderedDict([('rmse', 2511.776730947409)])                  
nb_trees=5743 val_loss=OrderedDict([('rmse', 2555.9992449127017)])                 
nb_trees=6201 val_loss=OrderedDict([('rmse', 2579.4215717657758)])                 
val_r2_score=-62.99097365909645                                                    
                                                                                   
LightGBM objective call #15 cur_best_score=0.00000
Params: bagging_fraction=0.9143155860180489 bagging_freq=15.0 feature_fraction=0.8549404898534534 lambda_l1=6.094738252790846 lambda_l2=5.4260924849244185 learning_rate=0.004143550425292757 ma

val_r2_score=-48.06808131378857                                                    
                                                                                   
LightGBM objective call #25 cur_best_score=0.00000
Params: bagging_fraction=0.8926498968598582 bagging_freq=15.0 feature_fraction=0.7515556194566675 lambda_l1=9.795163123774673 lambda_l2=2.355519334275221 learning_rate=0.0018561305928347662 max_bin=113.0 min_data_in_leaf=62.0 min_sum_hessian_in_leaf=2.8576592022234424 num_leaves=88.0
nb_trees=1066 val_loss=OrderedDict([('rmse', 2510.7035012776378)])                 
nb_trees=1185 val_loss=OrderedDict([('rmse', 2555.7627364635787)])                 
nb_trees=1057 val_loss=OrderedDict([('rmse', 2578.1744237478138)])                 
val_r2_score=-46.084346636055365                                                   
                                                                                    
LightGBM objective call #26 cur_best_score=0.00000
Params: bagging_fraction

nb_trees=1832 val_loss=OrderedDict([('rmse', 2555.775689148702)])                   
nb_trees=1692 val_loss=OrderedDict([('rmse', 2578.602945586642)])                   
val_r2_score=-48.226554204838266                                                    
                                                                                    
LightGBM objective call #36 cur_best_score=0.00000
Params: bagging_fraction=0.791030850955777 bagging_freq=12.0 feature_fraction=0.8975401949006084 lambda_l1=5.1758865987256355 lambda_l2=9.824950126108586 learning_rate=0.0023232513893085517 max_bin=103.0 min_data_in_leaf=25.0 min_sum_hessian_in_leaf=1.7490463954234527 num_leaves=11.0
nb_trees=3419 val_loss=OrderedDict([('rmse', 2513.4900415477377)])                  
nb_trees=2784 val_loss=OrderedDict([('rmse', 2557.469003460902)])                   
nb_trees=2988 val_loss=OrderedDict([('rmse', 2581.407969168624)])                   
val_r2_score=-63.465356572425335                                     

KeyboardInterrupt: 